In [2]:
"""This jupyter notebook section provides a sample usage of the protocol."""

import math
import plotly.figure_factory as ff
import content.helper.helper as helper
from plotly.offline import init_notebook_mode, iplot
from content.encryption.encryption import Encryption
from content.analyzers.key_analyzer import KeyAnalyzer

# Init the notebook for usage of Plotly.
init_notebook_mode(connected=True)

# Ask user for a message to encrypt and the desired cube side length.
message = input("Please type in the message you want to encrypt.\n")
cube_side_length = int(input("\nPlease type in the desired cube side length.\n"))

# Initialize the protocol.
protocol = Encryption(message=message, cube_side_length=cube_side_length)

# Print the layout of the cube.
print("\nThe layout of the desired cube is: ")
print(helper.get_cube_layout(cube_side_length=cube_side_length))

# Ask the user for a desired key length and then generate the key.
key_length = int(input("\nPlease type in the desired key length.\n"))
key = helper.generate_random_keys(
    length=key_length, max_index=math.floor(cube_side_length / 2)
)

# Print the random generated key.
print("\nThe random generated key is: ")
iplot(ff.create_table(helper.get_key_table(key=key)))

# Perform the key analyzing and print the new key again.
key = KeyAnalyzer(key=key).analyze()
print(f"\nThe reduced key has length of {len(key)}, and it is: ")
iplot(ff.create_table(helper.get_key_table(key=key)))

# Show user the current binary (The one after XOR operation).
print(f"\nThe plain-text in binary padded is: \n{protocol.get_current_binary()}\n")

# Encrypt the message based on the given key and show the encrypted binary.
protocol.encrypt(key=key)
print(f"\nThe cipher-text in binary is: \n{protocol.get_current_binary()}\n")

# Decrypt the message and show the plain-text recovered.
print(f"\nThe decrypted message is: {protocol.get_decrypted_str()}")


Please type in the message you want to encrypt.
Hi, this is Weiqi speaking.

Please type in the desired cube side length.
3

The layout of the desired cube is: 
    L1  C  R1
T1   0  0   0
C    0  0   0
D1   0  0   0

Please type in the desired key length.
25

The random generated key is: 



The reduced key has length of 22, and it is: 



The plain-text in binary padded is: 
100111100101100100010100010111100110001100000101100100101001101100100011000111101010101111111001110001011001110101100011000000111000011111100001011101101101111110111110100000000011011101111101100011011001100100101111100001010011001010000101000011101111101010111011011110000001001011010001000101101011101011001111111001101011110100111010010110010011100111001111110110001100011111100100010011000111101000000010110000101001100101000101001010111110101011111011110111110111010001010010000001100001011011000100001010011000011111001101011011110010011010111110101011111011110111110111010001010010000001100001011011000100001010011000011111001101011011110010


The cipher-text in binary is: 
1001101001000101111110111111111000001111000100010001010001111111101100010100100010001001101001001110000001001000110101101011000101111110101100101010010011101111101100111110111111011011011100110001011101000100010011001111000000011010000011010101100100101001110101010010111110101011

In [5]:
"""Analyze the reduced length of random generated keys."""

import numpy as np
from content.helper.helper import generate_random_keys
from content.analyzers.key_analyzer import KeyAnalyzer
# ------------------------- Results -----------------------------
# When key length is 20, the reduced key length is around: 15.05.
# When key length is 21, the reduced key length is around: 15.79.
# When key length is 25, the reduced key length is around: 18.73.
# When key length is 26, the reduced key length is around: 19.50.
# When key length is 27, the reduced key length is around: 20.20.
# When key length is 28, the reduced key length is around: 20.90.

# ------------------- The analyze process -----------------------
# Find the reduced key length in average.
length = 27
key_length = [
    len(KeyAnalyzer(key=generate_random_keys(length=length, max_index=1)).analyze())
    for _ in range(1000)
]
# Print the mean.
print(f"Key with length {length} will be reduced to length of {np.mean(key_length)}.")


Key with length 27 will be reduced to length of 20.157.


In [7]:
"""Analyze the reduced length of random generated keys."""

from itertools import chain
from content.analyzers.cubie_location_analyzer import CubieLocationAnalyzer

# Initialize the location analyzer with desired cube size and track location.
step = 5
location = 1
init_analyzer = CubieLocationAnalyzer(
    cube_side_length=3, track_item_location=location
)

# Get the initial locations.
locations = init_analyzer.get_all_location()

# Keep expanding the locations for more steps.
for _ in range(step - 1):
    locations_lists = [
        CubieLocationAnalyzer(
            cube_side_length=3, 
            track_item_location=location
        ).get_all_location()
        for location in locations
    ]
    
    # Make the list a set to remove duplicates.
    locations = set(chain.from_iterable(locations_lists))

print(f"Bit at location {location} can reach {len(locations)} different locations after {step} steps.")


Bit at location 1 can reach 216 different locations after 5 steps.


In [11]:
"""Track location of a certain bit after a list of moves."""

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from content.helper.helper import generate_random_keys
from content.analyzers.cubie_location_analyzer import CubieLocationAnalyzer

# Init the notebook for usage of Plotly.
init_notebook_mode(connected=True)

# Setup the analyzer and keys.
analyzer = CubieLocationAnalyzer(cube_side_length=3, track_item_location=0)
keys = generate_random_keys(length=40, max_index=1)

# Get locations and create the trace for scatter plot.
locations = analyzer.location_tracker(keys=keys)
trace = go.Scatter(
    x = [index + 1 for index in range(len(locations))],
    y = locations,
    mode = 'lines+markers'
)

# Plot the scatter plot in the notebook.
iplot([trace])


In [13]:
"""Track location of two bits after a list of moves."""

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from content.helper.helper import generate_random_keys
from content.analyzers.cubie_location_analyzer import CubieLocationAnalyzer

# Init the notebook for usage of Plotly.
init_notebook_mode(connected=True)

# Setup the analyzers and keys.
analyzer1 = CubieLocationAnalyzer(cube_side_length=3, track_item_location=0)
analyzer2 = CubieLocationAnalyzer(cube_side_length=3, track_item_location=1)
keys = generate_random_keys(length=20, max_index=1)

# Get locations and create the traces for scatter plot.
locations1 = analyzer1.location_tracker(keys=keys)
locations2 = analyzer2.location_tracker(keys=keys)

# Create a trace
trace1 = go.Scatter(
    x = [index + 1 for index in range(len(locations1))],
    y = locations1,
    mode = 'lines+markers'
)

# Create a trace
trace2 = go.Scatter(
    x = [index + 1 for index in range(len(locations2))],
    y = locations2,
    mode = 'lines+markers'
)

# Plot the scatter plot in the notebook.
iplot([trace1, trace2])
